In [13]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import dataloader
import omr
import os
import matplotlib.pyplot as plt
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
#PATH = "C:\\Users\\hitts\\Documents\\GitHub\\piano"
PATH = "C:\\Users\\hitts\\Desktop"
BATCH_SIZE = 0
VAL_SIZE = 0.2

In [ ]:
dl = dataloader.DataLoader(path=PATH, batch_size=BATCH_SIZE, val_size=VAL_SIZE, shuffle=False, verbose=True)
x_train, y_train, x_test, y_test = dl.get_ds()

print(x_train.shape)
print(y_train.shape)

In [ ]:
rand_idx = np.random.randint(len(x_train))
image = x_train[rand_idx]
label = y_train[rand_idx]
plt.imshow(image, cmap='gray', vmin=0, vmax=255)

In [ ]:
alphabetic_label = dl.decode_numerical_label(label)
print(alphabetic_label)

In [ ]:
model = omr.get_model(len(dl.get_alphabet()))
print(model.summary())

In [ ]:
model.fit(x_train, y_train, batch_size=2, epochs=5)

# CTC Loss Function Example

In [ ]:
batch_size = 8
num_labels = 6
max_label_length = 5
num_frames = 12
labels = tf.random.uniform([batch_size, max_label_length], minval=1, maxval=num_labels, dtype=tf.int32)
labels

In [ ]:
logits = tf.random.uniform([num_frames, batch_size, num_labels])
logits.shape

In [ ]:
label_length = tf.random.uniform([batch_size], minval=2, maxval=max_label_length, dtype=tf.int32)
label_length

In [ ]:
label_mask = tf.sequence_mask(label_length, maxlen=max_label_length, dtype = label_length.dtype)
label_mask

In [ ]:
labels *= label_mask
labels

In [ ]:
logit_length = [num_frames] * batch_size
logit_length

In [ ]:
with tf.GradientTape() as t:
    t.watch(logits)
    ref_loss = tf.nn.ctc_loss(
        labels = labels,
        logits = logits,
        label_length = label_length,
        logit_length = logit_length,
        blank_index = 0
    )
    print(ref_loss)
    print(tf.reduce_mean(ref_loss))

# CTC Loss Function Network

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

# Generate synthetic data (replace this with your data)
num_samples = 1000
img_height = 28
img_width = 28
num_classes = 10
max_text_length = 5

# Create random image data and labels
image_data = np.random.rand(num_samples, img_height, img_width, 1).astype(np.float32)
labels = np.random.randint(0, num_classes, size=(num_samples, max_text_length)).astype(np.int32)

# Make labels sparse
sparse_labels = []
for label in labels:
    sparse_label = [x for digit in label for x in [digit, 0]]
    sparse_label.pop()
    sparse_labels.append(sparse_label)
sparse_labels = np.array(sparse_labels)

# Define the model
input_img = layers.Input(shape=(img_height, img_width, 1))
conv_1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
maxpool_1 = layers.MaxPooling2D((2, 2))(conv_1)
conv_2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(maxpool_1)
maxpool_2 = layers.MaxPooling2D((2, 2))(conv_2)

# Convert the 2D feature maps to a sequence
reshaped = layers.Reshape((-1, 64))(maxpool_2)
lstm = layers.LSTM(128, return_sequences=True)(reshaped)
output = layers.Dense(num_classes + 1, activation='softmax')(lstm)  # +1 for CTC blank label

test_model = Model(inputs=input_img, outputs=output)

# CTC loss function
def ctc_loss(y_true, y_pred):
    # Reshape y_pred to have shape (batch_size, time_steps, num_classes + 1)
    input_length = tf.ones(shape=(tf.shape(y_true)[0],), dtype=tf.int32) * max_text_length
    label_length = tf.math.count_nonzero(y_true, axis=-1, keepdims=False, dtype=tf.int32)
    
    loss = tf.nn.ctc_loss(y_true, y_pred, input_length, label_length, logits_time_major=False)
    return tf.reduce_mean(loss)

# Compile the model with CTC loss
test_model.compile(optimizer='adam', loss=ctc_loss, metrics=[accuracy])

# Train the model
test_model.fit(image_data, labels, epochs=5, batch_size=32)

Epoch 1/5


TypeError: in user code:

    File "C:\Users\hitts\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hitts\AppData\Local\Temp\ipykernel_1088\4060988590.py", line 100, in accuracy  *
        y_true_padded = tf.pad(y_true_split, paddings=[[0, max_length - tf.shape(y_true_split)[0]]])

    TypeError: Failed to convert elements of tf.RaggedTensor(values=Tensor("StringsByteSplit/StringSplit:1", shape=(None,), dtype=string), row_splits=Tensor("StringsByteSplit/RaggedFromValueRowIds/RowPartitionFromValueRowIds/concat:0", shape=(None,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.
